In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np


/var/folders/lq/0q3fzmb57s3c187k6gjyx3_00000gn/T/ipykernel_94654/1282688944.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [26]:
#Keys for best data (not yet known for certain, placeholders for now)
norm_key = norm_key= 'N_6H'
raw_key = '6DB_m5.0eps8000'

In [38]:
def best_locations (norm_key, raw_key):
    '''Input: dictionary keys for accessing the cluster labels for the best raw and normalized clusterings
    Output: dictionary with the location and label data for the best clusterings'''

    norm = pd.read_csv('../Merging/Merge_Norm.csv')
    raw = pd.read_csv('../Merging/Merge_Raw.csv')
    
    with open('Norm_Plant_Clusters.p', 'rb') as f:
        norm_dict = pickle.load(f)
    
    with open('Raw_Plant_Clusters.p', 'rb') as f:
        raw_dict = pickle.load(f)

    norm = norm[['plantCode', 'Longitude', 'Latitude']]
    raw = raw[['plantCode', 'Longitude', 'Latitude']]

    norm_location = pd.merge(left=norm, right=norm_dict[norm_key], how='left', left_on='plantCode', right_on='plantCode').dropna()
    raw_location = pd.merge(left=raw, right=raw_dict[raw_key], how='left', left_on='plantCode', right_on='plantCode')


    return norm_location, raw_location

In [39]:
#norm and raw are dfs with results with gas consumption data (for line graph visuals)
#norm_location and raw_location are the raw and normalized results with just location info and labels
#(for location visuals)
norm_location, raw_location =  best_data (norm_key, raw_key)
norm_location

,plantCode,Longitude,Latitude,N_0,N_1,N_2,N_3,N_4,N_5
0,57950,-82.619167,43.266389,0.0,1.0,0.0,0.0,0.0,0.0
1,57949,-83.447222,43.741111,1.0,0.0,0.0,0.0,0.0,0.0
2,54867,-86.644300,46.409400,0.0,1.0,0.0,0.0,0.0,0.0
3,58427,-84.551667,42.719722,0.0,1.0,0.0,0.0,0.0,0.0
4,62192,-82.479331,42.773716,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
497,50466,-83.525300,30.068800,0.0,1.0,0.0,0.0,0.0,0.0
499,10275,-81.600600,27.911400,0.0,1.0,0.0,0.0,0.0,0.0
500,7699,-81.849446,27.746369,0.0,1.0,0.0,0.0,0.0,0.0
501,10062,-80.356600,25.835600,0.0,1.0,0.0,0.0,0.0,0.0
